In [1]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import numpy as np

In [2]:
df = pd.read_csv('miem_data.csv')
df.head()

,num,title,type
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР
2,3.0,Разработка программного комплекса решения эндш...,НИР
3,4.0,Программа объединения точек платежей для проти...,Прогр.
4,5.0,Антифрод (antifraud),Прогр.


In [4]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from pymystem3 import Mystem
mystem = Mystem() 

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [6]:
import string
punctuation = string.punctuation

In [7]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]   
    text = " ".join(tokens)   
    return text

In [8]:
from tqdm import tqdm

tqdm.pandas()
df['title_lemmatized'] = df['title'].progress_apply(preprocess_text)

100%|██████████| 253/253 [00:01<00:00, 209.01it/s]


In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [12]:
docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.title_lemmatized)]

In [13]:
model = Doc2Vec(vector_size=64, min_count=1, epochs=22)
model.build_vocab(docs)

In [14]:
model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)

In [15]:
card2vec = [model.infer_vector((df['title_lemmatized'][i].split(' '))) for i in range(0,len(df['title_lemmatized']))]

In [16]:
vec = np.array(card2vec).tolist()
df['vectors'] = vec
df.head()

,num,title,type,title_lemmatized,vectors
0,1.0,Изучение транспорта заряженных коллоидных част...,НИР,изучение транспорт заряжать коллоидный частица...,"[0.021411802619695663, -0.012915868312120438, ..."
1,2.0,Изучение конвекции Марангони и транспорта колл...,НИР,изучение конвекция марангонь транспорт коллоид...,"[0.03792756423354149, -0.017522942274808884, 0..."
2,3.0,Разработка программного комплекса решения эндш...,НИР,разработка программный комплекс решение эндшпи...,"[0.027339119464159012, -0.0043623377569019794,..."
3,4.0,Программа объединения точек платежей для проти...,Прогр.,программа объединение точка платеж противодейс...,"[0.014935723505914211, -0.006338078528642654, ..."
4,5.0,Антифрод (antifraud),Прогр.,антифрод antifraud,"[0.013650435023009777, -0.00017175286484416574..."


In [18]:
df.to_csv('miem_embeddings.csv', index=False) 